# Dekoratory

In [12]:
import functools

def shouter(func):

    @functools.wraps(func)
    def _shouter(*args, **kwargs):
        print('-- Before call: ', func.__name__)
        result = func(*args, **kwargs)
        print('-- After call: ', func.__name__)
        return result
    
    return _shouter

In [13]:
from typing import Callable
from typing import Any


def disable(func):
    
    @functools.wraps(func)
    def _disable(*args, **kwargs):
        pass
    return _disable

In [14]:
@shouter
@disable
def foo(name: str):
    """foo function"""
    print(f"foo-{name}")

#foo = shouter(disable(foo)) # interpreter works like this

In [15]:
foo("Hello")

-- Before call:  foo
-- After call:  foo


In [16]:
foo.__doc__

'foo function'

In [17]:
foo.__name__

'foo'

# Dekoratory z argumentami

In [18]:
def tag(tagname): # factory of tag decorators

    def tag_decorator(func):
        
        @functools.wraps(func)
        def _tag_decorator(*args, **kwargs):
            tag_before = f"<{tagname}>"
            tag_after = f"</{tagname}>"
            return tag_before + func(*args, **kwargs) + tag_after
        
        return _tag_decorator
    
    return tag_decorator

In [19]:
@tag("div")
@tag("h1")
def get_html(content):
    return content

# get_text = tag("h1")(get_text)

In [20]:
get_html("Text")

'<div><h1>Text</h1></div>'

In [21]:
get_html("Header")

'<div><h1>Header</h1></div>'

In [22]:
def repeat(count):
    
    def repeat_decorator(func):

        @functools.wraps(func)
        def _repeat_decorator(*args, **kwargs):
            for _ in range(count):
                func(*args, **kwargs)
        
        return _repeat_decorator
    
    return repeat_decorator

In [23]:
@repeat(3)
def foo():
    print('foo()')

In [24]:
foo()

foo()
foo()
foo()


# Implementacja dekoratora za pomocą klasy

In [76]:
class Debug:
    def __init__(self, func: Callable) -> None:
        self.function = func
        functools.update_wrapper(self, func)

    def __call__(self, *args, **kwargs):
        result = self.function(*args, **kwargs)
        name = self.function.__name__

        print(f"{name}({args!r}, {kwargs!r}): {result}")

In [77]:
@Debug
def add(a, b):
    return a + b

# add = Debug(add)

In [79]:
add(8, 45)

add((8, 45), {}): 53


In [65]:
class Repeat:
    def __init__(self, count):
        self.count = count

    def __call__(self, func):

        @functools.wraps(func)
        def _repeat_decorator(*args, **kwargs):
            for _ in range(self.count):
                func(*args, **kwargs)
        
        return _repeat_decorator

In [69]:
@Repeat(3)
@repeat(2)
def bar():
    print('bar()')

In [70]:
bar()

bar()
bar()
bar()
bar()
bar()
bar()


In [71]:
bar.__name__

'bar'

# Rejestracja przy pomocy dekoratorów

In [26]:
import collections


class EventRegistry:
    def __init__(self) -> None:
        self.registry = collections.defaultdict(list)

    def on(self, *events):
        def on_decorator(func: Callable) -> Callable:
            for event in events:
                self.registry[event].append(func)
            return func
        return on_decorator
    
    def fire(self, event: str, *args, **kwargs):
        for func in self.registry[event]:
            func(*args, **kwargs)

In [27]:
events = EventRegistry()

@events.on('start', 'success')
def start(service_name: str):
    print(f'Starting service {service_name}')

#start = events.on('start', 'success')(start)

@events.on('error')
def teardown():
    print('Shutting down system')


events.fire('start',"Start me UP!!!")
events.fire('error')

Starting service Start me UP!!!
Shutting down system
